In [41]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head(15)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,10556855,MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION...,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,10558440,FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRI...,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,10566033,ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,10570430,ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,10571689,INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [42]:
# removing SPECIAL_CONSIDERATIONS column
application_df_clean = application_df.drop(["EIN","NAME","SPECIAL_CONSIDERATIONS"], axis=1)

In [43]:
application_df_clean['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [44]:
# Changing the cutoff point to start from T25
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T25","T14","T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [45]:
application_df_clean["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [46]:
application_df_clean["CLASSIFICATION"].value_counts()[application_df_clean["CLASSIFICATION"].value_counts() >=58]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
Name: CLASSIFICATION, dtype: int64

In [47]:
# Changing the cutoff point to start from C1200
# use the variable name `application_types_to_replace`
application_types_to_replace = ["C1000","C2000","C1200"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()

Other    28237
C3000     1918
C2100     1883
C7000      777
C1700      287
         ...  
C4120        1
C2561        1
C1245        1
C8210        1
C2150        1
Name: CLASSIFICATION, Length: 69, dtype: int64

In [48]:
application_df_clean_numeric = pd.get_dummies(data=application_df_clean, 
                                        columns=["APPLICATION_TYPE", 
                                                 "AFFILIATION", 
                                                 "CLASSIFICATION", 
                                                 "USE_CASE", 
                                                 "ORGANIZATION",
                                                 "INCOME_AMT",
#                                                  "SPECIAL_CONSIDERATIONS"
                                                 
                                                 
                                                 ])

application_df_clean_numeric

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [49]:
# Split our preprocessed data into our features and target arrays
y = application_df_clean_numeric["IS_SUCCESSFUL"].values
x = application_df_clean_numeric.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [50]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scale = X_scaler.transform(X_train)
X_test_scale = X_scaler.transform(x_test)

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_layer1 =  100
hidden_layer2 = 120
hidden_layer3 = 80

neural = tf.keras.models.Sequential()

# First hidden layer
neural.add(
    tf.keras.layers.Dense(units=hidden_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
neural.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

# adding a third hidden layer
neural.add(tf.keras.layers.Dense(units=hidden_layer2, activation="sigmoid"))

# Output layer
neural.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
neural.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 100)               10900     
                                                                 
 dense_21 (Dense)            (None, 120)               12120     
                                                                 
 dense_22 (Dense)            (None, 120)               14520     
                                                                 
 dense_23 (Dense)            (None, 1)                 121       
                                                                 
Total params: 37,661
Trainable params: 37,661
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Compile the model
neural.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [62]:
# Train the model
train_model = neural.fit(X_train_scale,y_train,epochs=25)

Epoch 1/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5740 - accuracy: 0.7172
Epoch 2/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5579 - accuracy: 0.7291
Epoch 3/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5559 - accuracy: 0.7291
Epoch 4/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5544 - accuracy: 0.7313
Epoch 5/25
804/804 [==============================] - 1s 1ms/step - loss: 0.5538 - accuracy: 0.7315
Epoch 6/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7320
Epoch 7/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7322
Epoch 8/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7319
Epoch 9/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7324
Epoch 10/25
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7310

In [65]:
# Evaluate the model using the test data
model_loss, model_accuracy = neural.evaluate(X_test_scale,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5563 - accuracy: 0.7300 - 263ms/epoch - 981us/step
Loss: 0.5563194751739502, Accuracy: 0.7300291657447815


In [64]:
neural.save("./AlphabetSoupCharity_Optimization.h5")